In [ ]:
#all H must be represented by the sum of Pauli strings

#toy example: H = 2I + X + 3Z

import cmath
import numpy as np
import random

In [ ]:
class HamiltonianBuilder():
    def __init__(self,coefficients = {}):
        identity = np.eye(2) #ones on main dianonal
        sigma_x = np.array([[0,1],[1,0]])
        sigma_y = np.array([[0,-1j],[1j,0]])
        sigma_z = np.array([[1,0],[0,-1]])

        self.pauli_matrices = {
            'i': identity,
            'x': sigma_x,
            'y': sigma_y,
            'z': sigma_z
        }

        self.zerocoefficients()
        self.reset_coefficients(coefficients)
        self._build()

    
    def zerocoefficients(self):
        self.coefficients = {
            'i': 0,
            'x': 0,
            'y': 0,
            'z': 0
        }

    def reset_coefficients(self,coefficients):
        #float values for each pauli string (sum)
        #randomized if not given

        self.zerocoefficients()
        if len(coefficients) <= 0:
            self.init_random_coefficients()
        else:
            assert all(key in self.coefficients.keys() for key in coefficients.keys()), \
                "Coefficients dict incorrectly formed"
            self.coefficients.update(coefficients)
        
        self._build()

    def _build(self):
        self.hamiltonian = np.zeros(shape=(2, 2)).astype('complex128')
        for key in self.coefficients:
            self.hamiltonian += self.coefficients[key] * \
                self.pauli_matrices[key]
            
    def matrix(self):
        return self.hamiltonian
    
    def init_random_coefficients(self):
        for k,v in self.coefficients.items():
            self.coefficients[k] = random.randint(-3,3)

            



In [ ]:
#random coord generator

h_builder = HamiltonianBuilder()

print(h_builder.coefficients)
print('\n')
print(h_builder.matrix())

In [ ]:
#or specify coefficients

coefficients = {
    'i': 2,
    'x': 1,
    'y': 0,
    'z': 3
}

h_builder.reset_coefficients(coefficients)

print(h_builder.coefficients)
print('\n')
print(h_builder.matrix())


In [ ]:
#from scratch python qubit class

import numpy as np
import cmath
import random
import math


class Qubit():
    def __init__(self, coefficents=(0, 0)):
        self.comp_0 = np.array([1, 0]) # |0⟩
        self.comp_1 = np.array([0, 1]) # |0⟩
        self.reinitialise_state(coefficents)

    # set the coefficients
    def reinitialise_state(self, coefficents=(0, 0)):
        c0, c1 = coefficents
        if c0 == 0 and c1 == 0:
            c0 = complex(random.random(), random.random())
            c1 = complex(random.random(), random.random())
        self.c0, self.c1 = self._normalise_coefficients(c0, c1)

    # since |c_0|^2 and |c_1|^2 are probabilities, they need to sum to 1
    def _normalise_coefficients(self, c0, c1):
        denom = math.sqrt(c0*np.conj(c0) + c1*np.conj(c1))
        return c0/denom, c1/denom

    @property
    def state_vector(self):
        return np.array([self.c0, self.c1])

    def pprint_state(self):
        """
        pretty prints the state
        also prints it in polar form (https://en.wikipedia.org/wiki/Bloch_sphere)
        """
        r_0, theta_0 = cmath.polar(self.c0)
        r_1, theta_1 = cmath.polar(self.c1)
        dphase = theta_1 - theta_0
        print(f'({self.c0:.2f})|0⟩ + ({self.c1:.2f})|1⟩ ≡ {r_0:.2f}|0⟩ + {r_1:.2f}⋅e^({dphase/math.pi:.2f}iπ)⋅|1⟩')

# we can initialise a random state (already done from object initialisation)
qubit = Qubit()
qubit.pprint_state()
print('\n')

# or we can initialise a state of our choosing
qubit.reinitialise_state((0.5, 0.5))
qubit.pprint_state()

In [ ]:
#normal way for none larger systems, H(psi) = E(psi)

conj_psi = np.conj(qubit.state_vector)
psi = qubit.state_vector

an_energy = np.matmul(conj_psi, np.matmul(h_builder.matrix(), psi))


def analytically_measure_observable(observable, state_vector):
    energy = np.matmul(
        np.conj(state_vector),
        np.matmul(observable, state_vector)
    )
    return energy

print(an_energy)

In [ ]:
#characteristic equation for eigenvalues: det(H - EI) = 0

eigenvalues = np.linalg.eigvals(h_builder.matrix())
min_eigenvalue = np.min(eigenvalues)

print(f'Energy of ground state for single-qubit example is {np.real(min_eigenvalue):.3f}')
print('\n')

In [ ]:
#NOW initializing to |0> for ansatz
#must be easily prepared quantum state
#dont want an exponentially increasing number of gates

class Qubit2(Qubit):
    def __init__(self):
        self.comp_0 = np.array([1, 0]) # |0⟩
        self.comp_1 = np.array([0, 1])
        self.reinitialise_state()

    def reinitialise_state(self):
        self.c0, self.c1 = self._normalise_coefficients(1, 0)

    def apply_op(self,op):
        self.c0, self.c1 = np.matmul(op, self.state_vector)

    def apply_ops(self,ops):
        for op in ops:
            self.apply_op(op)

    
qubit = Qubit2()
qubit.pprint_state()

In [ ]:
#Rotation operators

def get_rx(theta):
    rx = np.array([
        [math.cos(theta/2), complex(0, -math.sin(theta/2))],
        [complex(0, -math.sin(theta/2)), math.cos(theta/2)]
    ])
    return rx

def get_ry(theta):
    rx = np.array([
        [math.cos(theta/2), -math.sin(theta/2)],
        [math.sin(theta/2), math.cos(theta/2)]
    ])
    return rx

ry = get_ry(math.pi/4)
rx = get_rx(-math.pi/2)
qubit.reinitialise_state()
qubit.apply_ops([ry, rx])
qubit.pprint_state()

#equal superposition of |0> and |1> with phase pi/4
#example for testing rotations


In [ ]:
#Real VQE!

class Qubit3(Qubit2):
    def __init__(self):
        super(Qubit3, self).__init__()

    def measure_state(self):
        prob_0 = self.c0 * np.conj(self.c0) #coeff expansion in QM

        #random measurment (assumed 50/50)
        if random.random() < prob_0: #collaspe to |0>
            self.c0 = complex(1,0)
            self.c1 = complex(0,0)
            return 1
        else: #collaspse to |1>
            self.c0 = complex(0,0)
            self.c1 = complex(1,0)
            return -1
        
    def estimate_energy(qubit,state_prep_ops,basis = 'z',shots = 1000):
        outcomes = []

        if basis == 'x':
            change_basis = get_ry(-math.pi/2)
        
        elif basis == 'y':
            change_basis = get_rx(math.pi/2)

        elif basis != 'z':
            raise ValueError(f'Invalid basis: {basis}')
        
        for _ in range(shots):
            qubit.reinitialise_state()
            qubit.apply_ops(state_prep_ops)
            if basis == 'x' or basis == 'y':
                qubit.apply_op(change_basis)

            outcomes.append(qubit.measure_state())
        
        return np.mean(outcomes)
    

h_builder = HamiltonianBuilder()
coefficients = {
    'i': 2,
    'x': 1,
    'y': 0,
    'z': 3
}

h_builder.reset_coefficients(coefficients)

qubit = Qubit3()
ansatz_gates = [get_ry(math.pi/4), get_rx(-math.pi/2)]
qubit.apply_ops(ansatz_gates)
analytical_result = analytically_measure_observable(h_builder.matrix(), qubit.state_vector)
print(f'Analytical result: {analytical_result:.2f}')
print('\n')




#VQE

exp_result = 2 #identity contribution

result = qubit.estimate_energy(ansatz_gates, basis = 'x',shots = 1000)
print(f" sigma_x estimate: {result:.2f}")
exp_result += result

result = qubit.estimate_energy(ansatz_gates, basis = 'z',shots = 1000)
print(f" sigma_z estimate: {3*result:.2f}")
exp_result += 3*result

print(f'VQE result: {exp_result:.2f}')







    



In [ ]:
#optimization

import scipy.optimize as optimize

class QuantumExperiment():
    def __init__ (self, h_builder, qubit, n_exp):
        self.h_builder = h_builder
        self.qubit = qubit
        self.n_exp = n_exp
        self.results = []

    def clear_results(self):
        self.results = []

    def evaluate(self,ansatz_params):
        thetay, thetax = ansatz_params
        ansatz_gates = [get_ry(thetay), get_rx(thetax)]
        exp_result = 0

        exp_result += self.h_builder.coefficients.get('i',0)

        for key in 'x','y','z':
            if key in self.h_builder.coefficients and self.h_builder.coefficients[key] > 0:
                weight = self.h_builder.coefficients[key]
                exp_result += weight * self.qubit.estimate_energy(ansatz_gates, basis = key, shots = self.n_exp)

        self.results.append(exp_result)
        return exp_result
    


    



In [ ]:
#run a quantum experiment

h_builder = HamiltonianBuilder()
coefficients = {
    'i': 2,
    'x': 1,
    'y': 0,
    'z': 3
}

h_builder.reset_coefficients(coefficients)
qubit = Qubit3()

q_exp = QuantumExperiment(h_builder, qubit, n_exp = 5000)

result = optimize.minimize(q_exp.evaluate,
                        (np.random.rand(2) - 0.5) * 4*math.pi, # randomly select initial point 
                        method='Powell',
                        tol=1e-3)

print(result)




In [ ]:
#analytical result

eigenvalues = np.linalg.eigvals(h_builder.matrix())
min_eigenvalue = np.min(eigenvalues)
print(f'Analytical result is {np.real(min_eigenvalue):.3f}')
print('\n')

In [ ]:
#python plotting

import matplotlib.pyplot as plt

length = len(q_exp.results)

print(length)

x_range = range(length)
plt.plot(x_range, q_exp.results)

#draw a horizontal line at the analytical result

plt.axhline(y=np.real(min_eigenvalue), color='r', linestyle='-')
plt.xlabel('Optimization Iteration')
plt.ylabel('Energy')
plt.title("classical optimization of VQE")
plt.legend(['VQE', 'Analytical'])
plt.show()


In [ ]:
#run at different iterations
i = 10
j = 0

q_exps = []

while(i<=1000000):
    q_exps.append(QuantumExperiment(h_builder, qubit, n_exp = i))
    result = optimize.minimize(q_exps[j].evaluate, 
                            (np.random.rand(2) - 0.5) * 4*math.pi, # randomly select initial point 
                            method='Powell',
                            tol=1e-3)
    
    analytical = np.real(min_eigenvalue)
    
    

    plot_range = len(q_exps[j].results)

    x_range = range(plot_range)

    plt.plot(x_range, q_exps[j].results)

    #draw a horizontal line at the analytical result

    plt.axhline(y=analytical, color='r', linestyle='-')

    plt.xlabel('Optimization Iteration')
    plt.ylabel('Energy')

    plt.title(f"classical optimization of VQE with {i} shots")

    plt.legend(['VQE', 'Analytical'])

    plt.show()

    i *= 10
    j += 1

#plotting


#this one is not made from qiskits backend
#hence the values below the analytical result




    
